In [1]:
Work.objects.count()

15796

In [2]:
def header(cols):
    print(r'\begin{tabular}{%s} \toprule' % ('c' * len(cols)))
    print(' & '.join(cols) + r'\\ \midrule')

def footer():
    print(r'\end{tabular}')

In [3]:
header(['\#', 'Work', '\# Ratings'])
MAX = 10
for rank, work in enumerate(Work.objects.popular()[:MAX], start=1):
    print(r'{} & {} & {}\\{}'.format(rank, work, work.nb_ratings, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccc} \toprule
\# & Work & \# Ratings\\ \midrule
1 & Death Note & 1453\\
2 & L'Attaque des Titans & 1330\\
3 & Le Voyage de Chihiro & 1196\\
4 & Naruto & 1167\\
5 & Princesse Mononoké & 1159\\
6 & Sword Art Online & 1141\\
7 & Fullmetal Alchemist: Brotherhood & 1138\\
8 & Fullmetal Alchemist & 1024\\
9 & Naruto: Shippuuden & 988\\
10 & Fairy Tail & 975\\ \bottomrule
\end{tabular}


In [5]:
from django.db.models import F, FloatField, ExpressionWrapper
from django.db.models.functions import Cast
from collections import Counter

def get_ratings(work):
    return Counter(work.rating_set.values_list('choice', flat=True))

'''header(['\#', 'Work', ''])
MAX = 10
for rank, work in enumerate(Work.objects.top().reverse().annotate(score=ExpressionWrapper(F('nb_likes') / F('nb_ratings'), output_field=FloatField()))[:MAX], start=1):
    print(r'{} & {} {}\\{}'.format(rank, work, get_ratings(work), r' \bottomrule' if rank == MAX else ''))
footer()'''

"header(['\\#', 'Work', ''])\nMAX = 10\nfor rank, work in enumerate(Work.objects.top().reverse().annotate(score=ExpressionWrapper(F('nb_likes') / F('nb_ratings'), output_field=FloatField()))[:MAX], start=1):\n    print(r'{} & {} {}\\{}'.format(rank, work, get_ratings(work), r' \x08ottomrule' if rank == MAX else ''))\nfooter()"

In [6]:
from collections import defaultdict

ratings = defaultdict(Counter)
for _id, title, choice in Rating.objects.values_list('work_id', 'work__title', 'choice'):
    ratings[(_id, title)][choice] += 1

In [7]:
for key in ratings.copy():
    if sum(ratings[key].values()) < 10:
        del ratings[key]

In [8]:
sum(ratings[(1, 'Death Note')].values())

1608

In [9]:
def nb_seen_ratings(key):
    return ratings[key]['like'] + ratings[key]['favorite'] + ratings[key]['dislike']

In [12]:
header(['\#', 'Work', '\# Fav', '\# Like', '\# Hate'])
MAX = 20
for rank, key in enumerate(sorted(ratings.keys(), key=lambda key:
        float('inf') if nb_seen_ratings(key) < 50 else
       (float(4*ratings[key]['favorite'] + 2*ratings[key]['like'] + (-0.1)*ratings[key]['neutral'] + (-2)*ratings[key]['dislike'])
        / sum(ratings[key].values())), reverse=False)[:MAX], start=1):
    # print(key, ratings[key]['dislike'])
    print(r'{} & {} & {} & {}\\{}'.format(rank, key[1], ratings[key]['favorite'] + ratings[key]['like'], ratings[key]['dislike'], r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccccc} \toprule
\# & Work & # Fav & # Like & # Hate\\ \midrule
1 & Dragon Ball GT & 96 & 161\\
2 & Glasslip & 24 & 36\\
3 & Girls Bravo: First Season & 26 & 39\\
4 & Captain Earth & 25 & 35\\
5 & Ai non stop! & 25 & 33\\
6 & Yu-Gi-Oh! GX & 128 & 158\\
7 & Astarotte's Toy & 31 & 38\\
8 & Choujigen Game Neptune: The Animation & 27 & 33\\
9 & Haruka Nogizaka's Secret & 24 & 27\\
10 & Dog Days & 57 & 66\\
11 & Wizard Barristers & 51 & 59\\
12 & .Hack - Le bracelet du crépuscule & 24 & 26\\
13 & Sakura Trick & 48 & 50\\
14 & Ikki Tousen & 35 & 39\\
15 & Rail Wars! & 60 & 56\\
16 & The Severing Crime Edge & 33 & 29\\
17 & To Love & 56 & 55\\
18 & Chaos;Head & 75 & 66\\
19 & Phi Brain: Puzzle of God & 30 & 23\\
20 & Tokyo Mew Mew & 48 & 40\\ \bottomrule
\end{tabular}


In [14]:
header(['\#', 'Work', '# Like', '# Hate'])
MAX = 10
for rank, work in enumerate(Work.objects.controversial()[:MAX], start=1):
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {} & {}\\{}'.format(rank, work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{cccc} \toprule
\# & Work & # Like & # Hate\\ \midrule
1 & School Days & 182 & 156\\
2 & Dragon Ball Z Movie 11: Bio-Broly & 122 & 108\\
3 & Rail Wars! & 58 & 56\\
4 & Chaos;Head & 73 & 66\\
5 & To Love & 52 & 55\\
6 & Freezing & 74 & 66\\
7 & Kiss x Sis & 121 & 96\\
8 & Sakura Trick & 46 & 50\\
9 & Yu-Gi-Oh! GX & 118 & 158\\
10 & Naruto the Movie 2: Legend of the Stone of Gelel & 114 & 89\\ \bottomrule
\end{tabular}


In [41]:
header(['\#', 'Work', '# Like', '# Hate'])
MAX = 20
for rank, work in enumerate(Work.objects.random().order_by('?')[:MAX], start=1):
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {} & {}\\{}'.format(rank, work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{cccc} \toprule
\# & Work & # Like & # Hate\\ \midrule
1 & Alice in borderland & 22 & 6\\
2 & Gekkan Shoujo Nozaki-kun Specials & 38 & 6\\
3 & Letter Bee & 34 & 9\\
4 & Star Blazers 2199 & 27 & 4\\
5 & High School DxD OVA & 19 & 5\\
6 & Yahari Ore no Seishun Love Comedy wa Machigatteiru. OVA & 31 & 3\\
7 & Dragon Ball Z Movie 04: Lord Slug & 17 & 2\\
8 & Gintama': Enchousen & 31 & 6\\
9 & UQ holder & 50 & 10\\
10 & Tsukimonogatari & 103 & 9\\
11 & Mes voisins les Yamada & 29 & 2\\
12 & His and Her Circumstances & 21 & 4\\
13 & School Rumble & 60 & 11\\
14 & Another: The Other & 33 & 8\\
15 & Dr Slump & 24 & 7\\
16 & Blue Exorcist: Runaway Kuro & 43 & 2\\
17 & Kannagi: Moshimo Kannagi ga Attara... & 19 & 1\\
18 & Death Note: Relight & 81 & 12\\
19 & Pokemon 3: The Movie & 79 & 13\\
20 & Nausicaä of the Valley of the Wind & 23 & 0\\ \bottomrule
\end{tabular}


In [48]:
ratings[(3236, 'Kaiba')]

Counter({'dislike': 4,
         'favorite': 8,
         'like': 34,
         'neutral': 12,
         'willsee': 27,
         'wontsee': 41})

In [23]:
kaiba = Work.objects.get(title='Kaiba')
madlax = Work.objects.get(title='Madlax')
fujiko = Work.objects.filter(title__icontains='Fujiko')[3]
momo = Work.objects.get(id=902)

In [26]:
header(['Work', '# Like', '# Hate'])
MAX = 20
for work in [kaiba, madlax, fujiko, momo]:
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {}\\{}'.format(work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccc} \toprule
Work & # Like & # Hate\\ \midrule
Kaiba & 34 & 4\\
Madlax & 19 & 4\\
Lupin the Third, The Woman Called Fujiko Mine & 14 & 1\\
A Letter to Momo & 42 & 1\\
\end{tabular}


In [21]:
get_ratings(fujiko)

Counter({'dislike': 1, 'like': 14, 'neutral': 5, 'willsee': 5, 'wontsee': 4})

In [22]:
get_ratings(momo)

Counter({'dislike': 1,
         'favorite': 2,
         'like': 42,
         'neutral': 13,
         'willsee': 31,
         'wontsee': 15})

In [60]:
get_ratings(kaiba)

Counter({'dislike': 4,
         'favorite': 8,
         'like': 34,
         'neutral': 12,
         'willsee': 27,
         'wontsee': 41})

In [61]:
get_ratings(Work.objects.get(title='Madlax'))

Counter({'dislike': 4,
         'favorite': 2,
         'like': 19,
         'neutral': 8,
         'willsee': 8,
         'wontsee': 18})

In [72]:
Work.objects.random().count()

901

In [93]:
Work.objects.filter(nb_dislikes__lte=1, nb_likes__gte=10).values_list('title')

<WorkQuerySet [('.hack//G.U. Trilogy',), ('Kurenai',), ('Roujin Z',), ('Kore wa Zombie Desu ka? of the Dead OVA',), ('Charlotte',), ('Blue Submarine No. 6',), ('Familiar of Zero F',), ('Ghost in the Shell: Arise - Border:3 Ghost Tears',), ('Hikaru no Go: Journey to the North Star Cup',), ('Spice and Wolf',), ('Heroic Age',), ('The Tatami Galaxy Specials',), ('Another Special',), ('RahXephon: Harmonic Convergence',), ('The Story of Saiunkoku',), ("Gen d'Hiroshima",), ('Yu Yu Hakusho: Eizou Hakusho II',), ('Ookiku Furikabutte: Natsu no Taikai Hen',), ('Kara no Kyoukai: Mirai Fukuin - Extra Chorus',), ('.hack//roots',), '...(remaining elements truncated)...']>

In [19]:
queryset = (Work.objects.exclude(nb_likes=0).annotate(
    dislike_rate=ExpressionWrapper(Cast(F('nb_dislikes'), FloatField()) / F('nb_likes'), output_field=FloatField()))
    .filter(nb_ratings__gte=30, nb_ratings__lte=126, dislike_rate__lte=5/42)
    .order_by('dislike_rate'))
print(queryset.count())
for work in queryset[:20]:
    print(work.id, work.title, work.nb_ratings, work.nb_likes, work.nb_dislikes, work.dislike_rate)

241
9038 GARO THE ANIMATION 31 22 0 0.0
10013 Le Garçon et la Bête 51 26 0 0.0
14205 Nausicaä of the Valley of the Wind 30 23 0 0.0
9071 Psycho-Pass Movie 48 36 0 0.0
9895 Psychic School Wars 32 15 0 0.0
1380 Kara no Kyoukai 6: Boukyaku Rokuon 82 54 1 0.0185185185185185
1304 Natsume's Book of Friends Sequel 62 46 1 0.0217391304347826
9814 Samurai X: Trust and Betrayal 58 45 1 0.0222222222222222
902 A Letter to Momo 58 42 1 0.0238095238095238
9863 True Tears 60 41 1 0.024390243902439
2913 Tasogare Otome x Amnesia: Taima Otome 53 40 1 0.025
1063 Moribito - Guardian of the Spirit 56 38 1 0.0263157894736842
3933 On Your Mark 51 36 1 0.0277777777777778
3428 Suisei no Gargantia Specials 49 34 1 0.0294117647058824
8736 Broken Blade 3 50 34 1 0.0294117647058824
10565 Genshiken 2 43 30 1 0.0333333333333333
9939 Eureka Seven - good night, sleep tight, young lovers 47 29 1 0.0344827586206897
2728 Now and Then, Here and There 41 29 1 0.0344827586206897
3187 Pale Cocoon 40 29 1 0.0344827586206897
1

In [62]:
for 

['Saki',
 'Elfen Lied: In the Passing Rain',
 'Le Fruit de la Grisaia',
 'Little Busters!',
 'Hokuto no Ken - Deluxe',
 'Rec',
 'Magical Girl Lyrical Nanoha',
 'Dream Eater Merry',
 'Kamisama Kiss',
 'Nura - Le seigneur des yokai',
 'Jormungand Season 2: Perfect Order',
 'Prison school',
 'Ranma ½',
 'Darker than black',
 'Kami nomi zo Shiru Sekai: Tenri-hen',
 'Evangelion - Neon genesis',
 'Planetes',
 'Kaiba',
 'Rainbow: Nisha Rokubou no Shichinin',
 'Gangsta']

In [71]:
get_ratings(Work.objects.filter(title__icontains='Fujiko')[3])

Counter({'dislike': 1, 'like': 14, 'neutral': 5, 'willsee': 5, 'wontsee': 4})

In [43]:
Work.objects.order_by('-nb_dislikes').values_list('title', 'nb_dislikes')

<WorkQuerySet [('Fairy Tail', 238), ('Naruto: Shippuuden', 226), ('Naruto', 211), ('Bleach', 178), ('Dragon Ball GT', 161), ('Yu-Gi-Oh! GX', 158), ('School Days', 155), ('Sword Art Online', 136), ('One Piece', 117), ('Dragon Ball Z Movie 11: Bio-Broly', 107), ('Kiss x Sis', 96), ('Rosario to Vampire', 96), ('Highschool of the Dead', 94), ('IS: Infinite Stratos', 92), ('Naruto the Movie 2: Legend of the Stone of Gelel', 89), ('Fairy Tail', 88), ('Vampire Knight', 82), ('Dragon Ball Z', 80), ('Angel Beats!', 74), ('Sword Art Online II', 73), '...(remaining elements truncated)...']>

<WorkQuerySet [<Work: Fullmetal Alchemist - Edition reliée>, <Work: FullMetal Alchemist>, <Work: Fullmetal Alchemist: Brotherhood>, <Work: Steins;Gate>, <Work: Princesse Mononoké>, <Work: Le Voyage de Chihiro>, <Work: Tokyo ghoul>, <Work: L'Attaque des Titans>, <Work: Death note>, <Work: Your Lie in April>]>

In [31]:
for work in Work.objects.top().reverse()[:10]:
    print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')

Girls Bravo: First Season 26 likes 39 dislikes
Astarotte's Toy 31 likes 38 dislikes
Captain Earth 25 likes 35 dislikes
Vividred Operation 13 likes 22 dislikes
Choujigen Game Neptune: The Animation 27 likes 33 dislikes
Ai non stop! 24 likes 33 dislikes
Haruka Nogizaka's Secret 24 likes 27 dislikes
Dragon Ball GT 87 likes 161 dislikes
Dog Days 55 likes 66 dislikes
Glasslip 23 likes 36 dislikes
